# Workflow for generating realizations of variability

This notebook generates ensemble realizations of variability based on the input dataset (SORRMv2.1 MPAS-Ocean model simulation).

In [ ]:
from aislens.generator import eof_decomposition, phase_randomization, generate_data
from aislens.config import config
import xarray as xr
import numpy as np
from pathlib import Path
import pickle

In [2]:
import matplotlib.pyplot as plt

In [ ]:
seasonality = xr.open_dataset(config.FILE_SEASONALITY_EXTRAPL, chunks={"Time": 36})
variability = xr.open_dataset(config.FILE_VARIABILITY_EXTRAPL, chunks={"Time": 36})

In [ ]:
# Verify that the time dimension in the dataset is named "time"
if 'Time' in variability.dims:
    variability = variability.rename({"Time": "time"})
if 'Time' in seasonality.dims:
    seasonality = seasonality.rename({"Time": "time"})

data = variability[config.SORRM_FLUX_VAR]

data_tmean = data.mean('time')
data_tstd = data.std('time')

data_norm = (data - data_tmean) / data_tstd

model, eofs, pcs, nmodes, varexpl = eof_decomposition(data_norm)

n_realizations = 3# config.N_REALIZATIONS

new_pcs = phase_randomization(pcs.values, n_realizations)

for i in range(n_realizations):
    new_data = generate_data(model, new_pcs, i, nmodes, 1)
    new_data = (new_data * data_tstd) + data_tmean
    new_data = xr.DataArray(new_data, dims=data.dims, coords=data.coords)
    new_data.attrs = data.attrs
    new_data.name = data.name

    forcing = seasonality + new_data
    forcing.to_netcdf(config.DIR_FORCINGS / f"forcing_realization_{i}.nc")

/Users/smurugan9/opt/anaconda3/envs/aislens/lib/python3.10/site-packages/dask/core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


In [ ]:
pickle_path = Path(config.DIR_PROCESSED) / "model.pkl"
with open(pickle_path, 'wb') as f:
    pickle.dump(model, f)
# Uncomment the following lines to load the model from a pickle file instead of running EOF above.
# with open(pickle_path, "rb") as f:
#     model = pickle.load(f)